In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime as dt
import numpy as np
now = dt.datetime.now() # current date and time
data = now.strftime("%Y_%m_%d")
mes_ano = now.strftime('%Y_%m')


In [2]:
# 1° PASSO
# USUARIOS CADASTRADOS / ENTRADAS NOVAS / BAIRROS ROTAS
# FUNÇÃO
##################################################################################################
entrada = pd.read_excel('ALMA_VIVA_CONT_12_04_2020.xlsx', sheet_name='DIRCEU') # não esquecer o sheet name
usuarios = pd.read_excel('bd_março.xlsx')
rotas_bairros = pd.read_excel('rotas_final.xlsx')
##################################################################################################

In [3]:
standard_header = ['matricula', 'nome','bairro','cidade', 'filial', 'status_frequencia','lat_long','zona']

In [4]:
usuarios.columns = standard_header

In [5]:
# TAGS DE ENTRADA
header_filtro = ['matricula', 'nome', 'rua', 'numero', 'complemento', 'bairro', 'cidade', 'telefone', 'filial','horario']

In [6]:
# LIMPEZA E UNIFORMIZAÇÃO DOS BAIRROS
entrada = entrada.reindex(columns=header_filtro)

In [7]:
import unicodedata
import re
def RemoverCaracteresEspeciais(palavra):
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

In [8]:
clean_bairros = []
for each in usuarios['bairro']:
    caracteres_especiais = RemoverCaracteresEspeciais(each)
    clean_bairros.append(caracteres_especiais)


In [9]:
usuarios['bairro'] = clean_bairros
usuarios['bairro'] = usuarios['bairro'].str.strip().str.upper().astype(str)

In [10]:
entrada['rua'] = entrada['rua'].str.replace('Q ', 'QUADRA ')
entrada['cidade'] = entrada['cidade'].str.upper().str.strip()
entrada['filial'] = entrada['filial'].str.upper().str.strip()
entrada['bairro'] = entrada['bairro'].astype(str)
entrada['matricula'] = entrada['matricula'].astype(int)

In [11]:
# 2° PASSO
# VERIFICAR FILIAL SEMPRE
################################################################################
filial = 'DIRCEU'
filtro_filial = entrada[entrada['filial'] == filial] # FILIAL
################################################################################ 
data = '2020_04_12_'
titulo_01 = data + filial +'_horario_01.xlsx'
titulo_02 = data + filial +'_horario_02.xlsx'
titulo_03 = data + filial +'_horario_03.xlsx'
################################################################################

In [12]:
# STATUS USUARIO
usuarios_ativos = usuarios[usuarios['status_frequencia'] == 'ATIVO']
usuarios_inativos = usuarios[usuarios['status_frequencia'] == 'INATIVO']

In [13]:
# ENTRADA NOVA FILTRO FILIAL/ATIVO
# ENTRADA NOVA FILTRO FILIAL/ATIVO
matricula_usuarios_ativos = usuarios_ativos['matricula'] # USUARIO ATIVOS NO CADASTRO FIXO
matricula_usuarios_inativos = usuarios_inativos['matricula'] # USUARIOS INATIVOS NO CADASTRO FIXO
matricula_entradas_filial = filtro_filial['matricula'] # ENTRADAS NOVAS POPR FILIAL
matricula_usuarios_todos = usuarios['matricula'] # TODAS AS ENTRADAS


In [14]:
# ENTRADAS ATIVAS
filtro_entradas_ativas = []
filtro_entradas_inativos_novos = []
for each in matricula_usuarios_ativos:
    for num in matricula_entradas_filial:
        if num == each:
            filtro_entradas_ativas.append(num)
            
print(filtro_entradas_ativas, len(filtro_entradas_ativas))

[237688, 185634, 81782, 251321, 139995, 239645, 225597, 254071, 197631, 170020, 220892, 225597, 125571, 243125, 220820, 220818, 230148, 251321, 220820, 197631, 230148] 21


In [15]:
matriculas_novas_inativas = list(set(matricula_entradas_filial) - set(filtro_entradas_ativas))

In [16]:
# ENTRADAS INATIVAS
filtro_entradas_inativas = []
for each in matricula_usuarios_inativos:
    for num in matriculas_novas_inativas:
        if num == each:
            filtro_entradas_inativas.append(num)
            
print(filtro_entradas_inativas, len(filtro_entradas_inativas))

[248476] 1


In [17]:
# ENTRADAS NOVAS
filtro_entradas_novas = list(set(matriculas_novas_inativas) - set(filtro_entradas_inativas))
print(filtro_entradas_novas, len(filtro_entradas_novas))
entradas_novas = entrada[entrada['matricula'].isin(filtro_entradas_novas)]
titulo_novos = data + '_novas_matriculas.xlsx'
entradas_novas.to_excel(titulo_novos)

[220808, 187517, 139888, 239909] 4


In [18]:
entradas_ativas_filial = entrada[entrada['matricula'].isin(filtro_entradas_ativas)]
print(len(entradas_ativas_filial))
filtro_entradas_ativas_filial = pd.DataFrame(entradas_ativas_filial)
filtro_entradas_ativas_filial['status_frequencia'] = 'ATIVO'
filtro_entradas_ativas_filial = filtro_entradas_ativas_filial.set_index('matricula')

16


In [19]:
entradas_novas_filial = entrada[entrada['matricula'].isin(filtro_entradas_novas)]
print(len(entradas_novas_filial))
filtro_entradas_novas_filial = pd.DataFrame(entradas_novas_filial)
filtro_entradas_novas_filial['status_frequencia'] = 'NOVO'
filtro_entradas_novas_filial = filtro_entradas_novas_filial.set_index('matricula')

4


In [20]:
entradas_inativas_filial = entrada[entrada['matricula'].isin(filtro_entradas_inativas)]
print(len(entradas_inativas_filial))
filtro_entradas_inativas_filial = pd.DataFrame(entradas_inativas_filial)
filtro_entradas_inativas_filial['status_frequencia'] = 'INATIVO'
filtro_entradas_inativas_filial = filtro_entradas_inativas_filial.set_index('matricula')

1


In [21]:
# DADOS DAS ENTRADAS NOVAS E ATIVAS CONCATENADAS
novos_ativos_inativos = pd.concat([filtro_entradas_novas_filial, filtro_entradas_ativas_filial,filtro_entradas_inativas_filial])
print('Total matriculas',len(novos_ativos_inativos))
ver_repetidos = novos_ativos_inativos


Total matriculas 21


In [22]:
# FILTRO MATRICULAS DUPLICADAS
duplicados_drop = novos_ativos_inativos.query('~index.duplicated()')
duplicados = novos_ativos_inativos.duplicated()
novos_ativos_inativos = duplicados_drop
if filial == 'DIRCEU':
    titulo_mat = mes_ano + '_dirceu_matriculas_mensal.xlsx'
    novos_ativos_inativos.to_excel(titulo_mat)
if filial == 'SÃO PEDRO':
    titulo_mat = mes_ano + '_sao_pedro_matriculas_mensal.xlsx'
    novos_ativos_inativos.to_excel(titulo_mat)


print('Total matriculas',len(duplicados_drop))

Total matriculas 21


In [23]:
clean_bairros_novos = []
for each in novos_ativos_inativos['bairro']:
    caracteres_especiais = RemoverCaracteresEspeciais(each)
    clean_bairros_novos.append(caracteres_especiais)
novos_ativos_inativos['bairro'] = clean_bairros_novos
novos_ativos_inativos['bairro'] = novos_ativos_inativos['bairro'].str.strip().str.upper()

In [24]:
if filial == 'SÃO PEDRO':
    novos_ativos_inativos.to_excel('novos_ativos_inativos_sao_pedro.xlsx')
elif filial == 'DIRCEU':
    novos_ativos_inativos.to_excel('novos_ativos_inativos_dirceu.xlsx')

In [25]:
# novos_ativos_inativos['horario'].sort_values().unique()

In [26]:
novos_ativos_inativos['horario'] = novos_ativos_inativos['horario'].astype(str).str.replace(':', '').str.replace('-','').str.replace('19000101 ', '').str.strip().astype(int)
horarios_filial = novos_ativos_inativos['horario']
horarios_filial.unique()

array([ 54500,   2000,  61000, 231000,  13500,   3000,      0, 232500,
       230000, 233500, 235000, 225000,   5500, 234500,    500])

In [27]:
# DIVISAO POR FILIAL
norte = rotas_bairros['NORTE']
sul = rotas_bairros['SUL']
leste = rotas_bairros['LESTE']
centro = rotas_bairros['CENTRO']
sdb = rotas_bairros['SDB']
sdc = rotas_bairros['SDC']
todos = rotas_bairros['TODAS']

In [28]:
# HORARIOS DE SAIDA

if filial == 'DIRCEU':
    dirceu_horario_01 = novos_ativos_inativos[(novos_ativos_inativos['horario'] >= 224000) & (novos_ativos_inativos['horario'] <= 233500)]
    dirceu_horario_02 = novos_ativos_inativos[((novos_ativos_inativos['horario'] > 233500) & (novos_ativos_inativos['horario'] < 240000)|(novos_ativos_inativos['horario'] >= 0) & (novos_ativos_inativos['horario'] <= 3500) )]
    dirceu_horario_03 = novos_ativos_inativos[(novos_ativos_inativos['horario'] > 3500) & (novos_ativos_inativos['horario'] <= 15500)]

    # COLETA
    dirceu_horario_04 = novos_ativos_inativos[(novos_ativos_inativos['horario'] > 50000) & (novos_ativos_inativos['horario'] <= 91000)] 
    print(' ', dirceu_horario_01.shape[0],' passageiros às 23:30\n ', dirceu_horario_02.shape[0],' passageiros às 00:30\n ', dirceu_horario_03.shape[0],' passageiros às 01:50\n ', dirceu_horario_04.shape[0],' coletas')
    
    # SEPARAÇÃO POR ROTA 01
    dirceu_horario_01_sem_rota = dirceu_horario_01[(~dirceu_horario_01['bairro'].isin(todos)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_norte = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(norte)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_sul = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(sul)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_leste = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(leste)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_centro = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(centro)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_sdb = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(sdb)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_sdc = dirceu_horario_01[(dirceu_horario_01['bairro'].isin(sdc)) & (dirceu_horario_01['cidade'] == 'TERESINA')]
    dirceu_horario_01_rota_timon = dirceu_horario_01[dirceu_horario_01['cidade'] == 'TIMON']
    soma_rotas = (dirceu_horario_01_rota_norte.shape[0],dirceu_horario_01_rota_sul.shape[0], dirceu_horario_01_rota_leste.shape[0], 
                  dirceu_horario_01_rota_centro.shape[0], dirceu_horario_01_rota_sdb.shape[0], dirceu_horario_01_rota_sdc.shape[0], 
                  dirceu_horario_01_rota_timon.shape[0], dirceu_horario_01_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ',sum(soma_rotas), 'passageiros foram distribuidos em rotas no primeiro horário')
    dropar = [ 'complemento', 'telefone', 'filial']
    dirceu_horario_01_sem_rota = dirceu_horario_01_sem_rota.drop(dropar , axis=1)
    dirceu_horario_01_rota_norte = dirceu_horario_01_rota_norte.drop(dropar , axis=1)
    dirceu_horario_01_rota_sul = dirceu_horario_01_rota_sul.drop(dropar , axis=1)
    dirceu_horario_01_rota_leste = dirceu_horario_01_rota_leste.drop(dropar , axis=1)
    dirceu_horario_01_rota_centro = dirceu_horario_01_rota_centro.drop(dropar , axis=1)
    dirceu_horario_01_rota_sdb = dirceu_horario_01_rota_sdb.drop(dropar , axis=1)
    dirceu_horario_01_rota_sdc = dirceu_horario_01_rota_sdc.drop(dropar , axis=1)
    dirceu_horario_01_rota_timon = dirceu_horario_01_rota_timon.drop(dropar , axis=1)
    

    # ADIÇÃO DE ASSINATURA E SUBTITULOS
    dirceu_horario_01_sem_rota['zona'] = 'DESCONHECIDA'
    dirceu_horario_01_sem_rota['assinatura'] = np.nan
    dirceu_horario_01_rota_norte['zona'] = 'NORTE'
    dirceu_horario_01_rota_norte['assinatura'] = np.nan
    dirceu_horario_01_rota_sul['zona'] = 'SUL'
    dirceu_horario_01_rota_sul['assinatura'] = np.nan
    dirceu_horario_01_rota_leste['zona'] = 'LESTE'
    dirceu_horario_01_rota_leste['assinatura'] = np.nan
    dirceu_horario_01_rota_centro['zona'] = 'CENTRO'
    dirceu_horario_01_rota_centro['assinatura'] = np.nan
    dirceu_horario_01_rota_sdb['zona'] = 'SDB'
    dirceu_horario_01_rota_sdb['assinatura'] = np.nan
    dirceu_horario_01_rota_sdc['zona'] = 'SDC'
    dirceu_horario_01_rota_sdc['assinatura'] = np.nan
    dirceu_horario_01_rota_timon['zona'] = 'TIMON'
    dirceu_horario_01_rota_timon['assinatura'] = np.nan
    
    # ORDEM
    ordem = ['nome', 'rua', 'numero', 'bairro', 'zona','horario','assinatura','status_frequencia']
    dirceu_horario_01_sem_rota = dirceu_horario_01_sem_rota[ordem]
    dirceu_horario_01_rota_sul = dirceu_horario_01_rota_sul[ordem]
    dirceu_horario_01_rota_norte = dirceu_horario_01_rota_norte[ordem]
    dirceu_horario_01_rota_leste = dirceu_horario_01_rota_leste[ordem]
    dirceu_horario_01_rota_centro = dirceu_horario_01_rota_centro[ordem]
    dirceu_horario_01_rota_sdb = dirceu_horario_01_rota_sdb[ordem]
    dirceu_horario_01_rota_sdc = dirceu_horario_01_rota_sdc[ordem]
    dirceu_horario_01_rota_timon = dirceu_horario_01_rota_timon[ordem]
    #  ARQUIVO DIRCEU HORARIO 01
    
    dirceu_rotas_horario_01 = pd.ExcelWriter(titulo_01, engine='xlsxwriter')
    dirceu_horario_01_rota_sul.to_excel(dirceu_rotas_horario_01, sheet_name='SUL')
    dirceu_horario_01_rota_norte.to_excel(dirceu_rotas_horario_01, sheet_name='NORTE')
    dirceu_horario_01_rota_leste.to_excel(dirceu_rotas_horario_01, sheet_name='LESTE')
    dirceu_horario_01_rota_centro.to_excel(dirceu_rotas_horario_01, sheet_name='CENTRO')
    dirceu_horario_01_rota_sdb.to_excel(dirceu_rotas_horario_01, sheet_name='SDB')
    dirceu_horario_01_rota_sdc.to_excel(dirceu_rotas_horario_01, sheet_name='SDC')
    dirceu_horario_01_rota_timon.to_excel(dirceu_rotas_horario_01, sheet_name='TIMON')
    dirceu_horario_01_sem_rota.to_excel(dirceu_rotas_horario_01, sheet_name='SEM_ROTA')
    dirceu_rotas_horario_01.save()
    
    # SEPARAÇÃO POR ROTA 02
    dirceu_horario_02_sem_rota = dirceu_horario_02[(~dirceu_horario_02['bairro'].isin(todos)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_norte = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(norte)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_sul = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(sul)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_leste = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(leste)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_centro = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(centro)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_sdb = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(sdb)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_sdc = dirceu_horario_02[(dirceu_horario_02['bairro'].isin(sdc)) & (dirceu_horario_02['cidade'] == 'TERESINA')]
    dirceu_horario_02_rota_timon = dirceu_horario_02[dirceu_horario_02['cidade'] == 'TIMON']
    
    soma_rotas = (dirceu_horario_02_rota_norte.shape[0],dirceu_horario_02_rota_sul.shape[0], dirceu_horario_02_rota_leste.shape[0], 
                  dirceu_horario_02_rota_centro.shape[0], dirceu_horario_02_rota_sdb.shape[0], dirceu_horario_02_rota_sdc.shape[0], 
                  dirceu_horario_02_rota_timon.shape[0], dirceu_horario_02_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ',sum(soma_rotas), 'passageiros foram distribuidos em rotas no segundo horário')
    
    dropar = [ 'complemento', 'telefone', 'filial']
    dirceu_horario_02_sem_rota = dirceu_horario_02_sem_rota.drop(dropar , axis=1)
    dirceu_horario_02_rota_norte = dirceu_horario_02_rota_norte.drop(dropar , axis=1)
    dirceu_horario_02_rota_sul = dirceu_horario_02_rota_sul.drop(dropar , axis=1)
    dirceu_horario_02_rota_leste = dirceu_horario_02_rota_leste.drop(dropar , axis=1)
    dirceu_horario_02_rota_centro = dirceu_horario_02_rota_centro.drop(dropar , axis=1)
    dirceu_horario_02_rota_sdb = dirceu_horario_02_rota_sdb.drop(dropar , axis=1)
    dirceu_horario_02_rota_sdc = dirceu_horario_02_rota_sdc.drop(dropar , axis=1)
    dirceu_horario_02_rota_timon = dirceu_horario_02_rota_timon.drop(dropar , axis=1)
    
    # ASSINATURA E SUBTITULO
    dirceu_horario_02_sem_rota['zona'] = 'DESCONHECIDA'
    dirceu_horario_02_sem_rota['assinatura'] = np.nan
    dirceu_horario_02_rota_norte['zona'] = 'NORTE'
    dirceu_horario_02_rota_norte['assinatura'] = np.nan
    dirceu_horario_02_rota_sul['zona'] = 'SUL'
    dirceu_horario_02_rota_sul['assinatura'] = np.nan
    dirceu_horario_02_rota_leste['zona'] = 'LESTE'
    dirceu_horario_02_rota_leste['assinatura'] = np.nan
    dirceu_horario_02_rota_centro['zona'] = 'CENTRO'
    dirceu_horario_02_rota_centro['assinatura'] = np.nan
    dirceu_horario_02_rota_sdb['zona'] = 'SDB'
    dirceu_horario_02_rota_sdb['assinatura'] = np.nan
    dirceu_horario_02_rota_sdc['zona'] = 'SDC'
    dirceu_horario_02_rota_sdc['assinatura'] = np.nan
    dirceu_horario_02_rota_timon['zona'] = 'TIMON'
    dirceu_horario_02_rota_timon['assinatura'] = np.nan
    
    
    # ORDEM
    dirceu_horario_02_sem_rota = dirceu_horario_02_sem_rota[ordem]
    dirceu_horario_02_rota_sul = dirceu_horario_02_rota_sul[ordem]
    dirceu_horario_02_rota_norte = dirceu_horario_02_rota_norte[ordem]
    dirceu_horario_02_rota_leste = dirceu_horario_02_rota_leste[ordem]
    dirceu_horario_02_rota_centro = dirceu_horario_02_rota_centro[ordem]
    dirceu_horario_02_rota_sdb = dirceu_horario_02_rota_sdb[ordem]
    dirceu_horario_02_rota_sdc = dirceu_horario_02_rota_sdc[ordem]
    dirceu_horario_02_rota_timon = dirceu_horario_02_rota_timon[ordem]
    
    #  ARQUIVO DIRCEU HORARIO 02
    
    dirceu_rotas_horario_02 = pd.ExcelWriter(titulo_02, engine='xlsxwriter')
    dirceu_horario_02_rota_sul.to_excel(dirceu_rotas_horario_02, sheet_name='SUL')
    dirceu_horario_02_rota_norte.to_excel(dirceu_rotas_horario_02, sheet_name='NORTE')
    dirceu_horario_02_rota_leste.to_excel(dirceu_rotas_horario_02, sheet_name='LESTE')
    dirceu_horario_02_rota_centro.to_excel(dirceu_rotas_horario_02, sheet_name='CENTRO')
    dirceu_horario_02_rota_sdb.to_excel(dirceu_rotas_horario_02, sheet_name='SDB')
    dirceu_horario_02_rota_sdc.to_excel(dirceu_rotas_horario_02, sheet_name='SDC')
    dirceu_horario_02_rota_timon.to_excel(dirceu_rotas_horario_02, sheet_name='TIMON')
    dirceu_horario_02_sem_rota.to_excel(dirceu_rotas_horario_02, sheet_name='SEM_ROTA')
    dirceu_rotas_horario_02.save()
    
    # SEPARAÇÃO POR ROTA 03
    dirceu_horario_03_sem_rota = dirceu_horario_03[(~dirceu_horario_03['bairro'].isin(todos)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_norte = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(norte)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_sul = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(sul)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_leste = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(leste)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_centro = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(centro)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_sdb = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(sdb)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_sdc = dirceu_horario_03[(dirceu_horario_03['bairro'].isin(sdc)) & (dirceu_horario_03['cidade'] == 'TERESINA')]
    dirceu_horario_03_rota_timon = dirceu_horario_03[dirceu_horario_03['cidade'] == 'TIMON']
    
    soma_rotas = (dirceu_horario_03_rota_norte.shape[0],dirceu_horario_03_rota_sul.shape[0], dirceu_horario_03_rota_leste.shape[0], 
                  dirceu_horario_03_rota_centro.shape[0], dirceu_horario_03_rota_sdb.shape[0], dirceu_horario_03_rota_sdc.shape[0], 
                  dirceu_horario_03_rota_timon.shape[0], dirceu_horario_03_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ',sum(soma_rotas), 'passageiros foram distribuidos em rotas no terceiro horário')
    
    dropar = ['complemento', 'telefone', 'filial']
    dirceu_horario_03_sem_rota = dirceu_horario_03_sem_rota.drop(dropar , axis=1)
    dirceu_horario_03_rota_norte = dirceu_horario_03_rota_norte.drop(dropar , axis=1)
    dirceu_horario_03_rota_sul = dirceu_horario_03_rota_sul.drop(dropar , axis=1)
    dirceu_horario_03_rota_leste = dirceu_horario_03_rota_leste.drop(dropar , axis=1)
    dirceu_horario_03_rota_centro = dirceu_horario_03_rota_centro.drop(dropar , axis=1)
    dirceu_horario_03_rota_sdb = dirceu_horario_03_rota_sdb.drop(dropar , axis=1)
    dirceu_horario_03_rota_sdc = dirceu_horario_03_rota_sdc.drop(dropar , axis=1)
    dirceu_horario_03_rota_timon = dirceu_horario_03_rota_timon.drop(dropar , axis=1)
    
    # ADIÇÃO DE ASSINATURA E SUBTITULOS
    dirceu_horario_03_sem_rota['zona'] = 'DESCONHECIDA'
    dirceu_horario_03_sem_rota['assinatura'] = np.nan
    dirceu_horario_03_rota_norte['zona'] = 'NORTE'
    dirceu_horario_03_rota_norte['assinatura'] = np.nan
    dirceu_horario_03_rota_sul['zona'] = 'SUL'
    dirceu_horario_03_rota_sul['assinatura'] = np.nan
    dirceu_horario_03_rota_leste['zona'] = 'LESTE'
    dirceu_horario_03_rota_leste['assinatura'] = np.nan
    dirceu_horario_03_rota_centro['zona'] = 'CENTRO'
    dirceu_horario_03_rota_centro['assinatura'] = np.nan
    dirceu_horario_03_rota_sdb['zona'] = 'SDB'
    dirceu_horario_03_rota_sdb['assinatura'] = np.nan
    dirceu_horario_03_rota_sdc['zona'] = 'SDC'
    dirceu_horario_03_rota_sdc['assinatura'] = np.nan
    dirceu_horario_03_rota_timon['zona'] = 'TIMON'
    dirceu_horario_03_rota_timon['assinatura'] = np.nan
    
    # ORDEM
    dirceu_horario_03_sem_rota = dirceu_horario_03_sem_rota[ordem]
    dirceu_horario_03_rota_sul = dirceu_horario_03_rota_sul[ordem]
    dirceu_horario_03_rota_norte = dirceu_horario_03_rota_norte[ordem]
    dirceu_horario_03_rota_leste = dirceu_horario_03_rota_leste[ordem]
    dirceu_horario_03_rota_centro = dirceu_horario_03_rota_centro[ordem]
    dirceu_horario_03_rota_sdb = dirceu_horario_03_rota_sdb[ordem]
    dirceu_horario_03_rota_sdc = dirceu_horario_03_rota_sdc[ordem]
    dirceu_horario_03_rota_timon = dirceu_horario_03_rota_timon[ordem]

    #  ARQUIVO DIRCEU HORARIO 03
    
    dirceu_rotas_horario_03 = pd.ExcelWriter(titulo_03, engine='xlsxwriter')
    dirceu_horario_03_rota_sul.to_excel(dirceu_rotas_horario_03, sheet_name='SUL')
    dirceu_horario_03_rota_norte.to_excel(dirceu_rotas_horario_03, sheet_name='NORTE')
    dirceu_horario_03_rota_leste.to_excel(dirceu_rotas_horario_03, sheet_name='LESTE')
    dirceu_horario_03_rota_centro.to_excel(dirceu_rotas_horario_03, sheet_name='CENTRO')
    dirceu_horario_03_rota_sdb.to_excel(dirceu_rotas_horario_03, sheet_name='SDB')
    dirceu_horario_03_rota_sdc.to_excel(dirceu_rotas_horario_03, sheet_name='SDC')
    dirceu_horario_03_rota_timon.to_excel(dirceu_rotas_horario_03, sheet_name='TIMON')
    dirceu_horario_03_sem_rota.to_excel(dirceu_rotas_horario_03, sheet_name='SEM_ROTA')
    dirceu_rotas_horario_03.save()

  9  passageiros às 23:30
  7  passageiros às 00:30
  2  passageiros às 01:50
  3  coletas
  9 passageiros foram distribuidos em rotas no primeiro horário
  7 passageiros foram distribuidos em rotas no segundo horário
  2 passageiros foram distribuidos em rotas no terceiro horário


In [29]:

if filial == 'SÃO PEDRO':
    sao_pedro_horario_01 = novos_ativos_inativos[
        (novos_ativos_inativos['horario'] >= 223000) & (novos_ativos_inativos['horario'] <= 240000) | (
                    novos_ativos_inativos['horario'] >= 0) & (novos_ativos_inativos['horario'] <= 500)]
    sao_pedro_horario_02 = novos_ativos_inativos[
        (novos_ativos_inativos['horario'] > 500) & (novos_ativos_inativos['horario'] <= 10000)]
    sao_pedro_horario_03 = novos_ativos_inativos[
        (novos_ativos_inativos['horario'] > 10000) & (novos_ativos_inativos['horario'] <= 22000)]
    
    # COLETA
    sao_pedro_horario_04 = novos_ativos_inativos[
        (novos_ativos_inativos['horario'] > 50000) & (novos_ativos_inativos['horario'] <= 82500)]
    print(' ',sao_pedro_horario_01.shape[0],' passageiros às 00:00\n ', sao_pedro_horario_02.shape[0],' passageiros às 01:00\n ', sao_pedro_horario_03.shape[0],' passageiros às 02:00\n ',
          sao_pedro_horario_04.shape[0], ' coletas'
         ,)

    # SEPARAÇÃO POR ROTA 01
    
    sao_pedro_horario_01_sem_rota = sao_pedro_horario_01[(~sao_pedro_horario_01['bairro'].isin(todos)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_norte = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(norte)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_sul = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(sul)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_leste = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(leste)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_centro = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(centro)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_sdb = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(sdb)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_sdc = sao_pedro_horario_01[(sao_pedro_horario_01['bairro'].isin(sdc)) & (sao_pedro_horario_01['cidade'] == 'TERESINA')]
    sao_pedro_horario_01_rota_timon = sao_pedro_horario_01[sao_pedro_horario_01['cidade'] == 'TIMON']
    # CONFERÊNCIA
    soma_rotas = (sao_pedro_horario_01_rota_norte.shape[0],sao_pedro_horario_01_rota_sul.shape[0], sao_pedro_horario_01_rota_leste.shape[0], 
                  sao_pedro_horario_01_rota_centro.shape[0], sao_pedro_horario_01_rota_sdb.shape[0], sao_pedro_horario_01_rota_sdc.shape[0], 
                  sao_pedro_horario_01_rota_timon.shape[0], sao_pedro_horario_01_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ',sum(soma_rotas), 'passageiros foram distribuidos em rotas no primeiro horário')
    # DROPAR
    dropar = [ 'complemento', 'telefone', 'filial']
    sao_pedro_horario_01_sem_rota = sao_pedro_horario_01_sem_rota.drop(dropar , axis=1)
    sao_pedro_horario_01_rota_norte = sao_pedro_horario_01_rota_norte.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_sul = sao_pedro_horario_01_rota_sul.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_leste = sao_pedro_horario_01_rota_leste.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_centro = sao_pedro_horario_01_rota_centro.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_sdb = sao_pedro_horario_01_rota_sdb.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_sdc = sao_pedro_horario_01_rota_sdc.drop(dropar, axis=1)
    sao_pedro_horario_01_rota_timon = sao_pedro_horario_01_rota_timon.drop(dropar, axis=1)

    # ADIÇÃO DE ASSINATURA E SUBTITULOS
    sao_pedro_horario_01_sem_rota['zona'] = 'DESCONHECIDA'
    sao_pedro_horario_01_sem_rota['assinatura'] = np.nan
    sao_pedro_horario_01_rota_norte['zona'] = 'NORTE'
    sao_pedro_horario_01_rota_norte['assinatura'] = np.nan
    sao_pedro_horario_01_rota_sul['zona'] = 'SUL'
    sao_pedro_horario_01_rota_sul['assinatura'] = np.nan
    sao_pedro_horario_01_rota_leste['zona'] = 'LESTE'
    sao_pedro_horario_01_rota_leste['assinatura'] = np.nan
    sao_pedro_horario_01_rota_centro['zona'] = 'CENTRO'
    sao_pedro_horario_01_rota_centro['assinatura'] = np.nan
    sao_pedro_horario_01_rota_sdb['zona'] = 'SDB'
    sao_pedro_horario_01_rota_sdb['assinatura'] = np.nan
    sao_pedro_horario_01_rota_sdc['zona'] = 'SDC'
    sao_pedro_horario_01_rota_sdc['assinatura'] = np.nan
    sao_pedro_horario_01_rota_timon['zona'] = 'TIMON'
    sao_pedro_horario_01_rota_timon['assinatura'] = np.nan
    
    # ORDEM
    ordem = ['nome', 'rua', 'numero', 'bairro', 'zona', 'horario', 'assinatura', 'status_frequencia']
    sao_pedro_horario_01_sem_rota = sao_pedro_horario_01_sem_rota[ordem]
    sao_pedro_horario_01_rota_sul = sao_pedro_horario_01_rota_sul[ordem]
    sao_pedro_horario_01_rota_norte = sao_pedro_horario_01_rota_norte[ordem]
    sao_pedro_horario_01_rota_leste = sao_pedro_horario_01_rota_leste[ordem]
    sao_pedro_horario_01_rota_centro = sao_pedro_horario_01_rota_centro[ordem]
    sao_pedro_horario_01_rota_sdb = sao_pedro_horario_01_rota_sdb[ordem]
    sao_pedro_horario_01_rota_sdc = sao_pedro_horario_01_rota_sdc[ordem]
    sao_pedro_horario_01_rota_timon = sao_pedro_horario_01_rota_timon[ordem]

    #  ARQUIVO sao_pedro HORARIO 01
    
    sao_pedro_rotas_horario_01 = pd.ExcelWriter(titulo_01, engine='xlsxwriter')
    sao_pedro_horario_01_rota_sul.to_excel(sao_pedro_rotas_horario_01, sheet_name='SUL')
    sao_pedro_horario_01_rota_norte.to_excel(sao_pedro_rotas_horario_01, sheet_name='NORTE')
    sao_pedro_horario_01_rota_leste.to_excel(sao_pedro_rotas_horario_01, sheet_name='LESTE')
    sao_pedro_horario_01_rota_centro.to_excel(sao_pedro_rotas_horario_01, sheet_name='CENTRO')
    sao_pedro_horario_01_rota_sdb.to_excel(sao_pedro_rotas_horario_01, sheet_name='SDB')
    sao_pedro_horario_01_rota_sdc.to_excel(sao_pedro_rotas_horario_01, sheet_name='SDC')
    sao_pedro_horario_01_rota_timon.to_excel(sao_pedro_rotas_horario_01, sheet_name='TIMON')
    sao_pedro_horario_01_sem_rota.to_excel(sao_pedro_rotas_horario_01, sheet_name='SEM ROTA')
    
    sao_pedro_rotas_horario_01.save()
    

    # SEPARAÇÃO POR ROTA 02
    sao_pedro_horario_02_sem_rota = sao_pedro_horario_02[(~sao_pedro_horario_02['bairro'].isin(todos)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_norte = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(norte)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_sul = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(sul)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_leste = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(leste)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_centro = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(centro)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_sdb = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(sdb)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_sdc = sao_pedro_horario_02[(sao_pedro_horario_02['bairro'].isin(sdc)) & (sao_pedro_horario_02['cidade'] == 'TERESINA')]
    sao_pedro_horario_02_rota_timon = sao_pedro_horario_02[sao_pedro_horario_02['cidade'] == 'TIMON']
    # CONFERÊNCIA
    soma_rotas = (sao_pedro_horario_02_rota_norte.shape[0],sao_pedro_horario_02_rota_sul.shape[0], sao_pedro_horario_02_rota_leste.shape[0], 
                  sao_pedro_horario_02_rota_centro.shape[0], sao_pedro_horario_02_rota_sdb.shape[0], sao_pedro_horario_02_rota_sdc.shape[0], 
                  sao_pedro_horario_02_rota_timon.shape[0], sao_pedro_horario_02_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ',sum(soma_rotas), 'passageiros foram distribuidos em rotas no segundo horário')
    # DROPAR COLUNAS
    sao_pedro_horario_02_sem_rota = sao_pedro_horario_02_sem_rota.drop(dropar , axis=1)
    sao_pedro_horario_02_rota_norte = sao_pedro_horario_02_rota_norte.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_sul = sao_pedro_horario_02_rota_sul.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_leste = sao_pedro_horario_02_rota_leste.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_centro = sao_pedro_horario_02_rota_centro.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_sdb = sao_pedro_horario_02_rota_sdb.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_sdc = sao_pedro_horario_02_rota_sdc.drop(dropar, axis=1)
    sao_pedro_horario_02_rota_timon = sao_pedro_horario_02_rota_timon.drop(dropar, axis=1)

    # ASSINATURA E SUBTITULO
    sao_pedro_horario_02_sem_rota['zona'] = 'DESCONHECIDA'
    sao_pedro_horario_02_sem_rota['assinatura'] = np.nan
    sao_pedro_horario_02_rota_norte['zona'] = 'NORTE'
    sao_pedro_horario_02_rota_norte['assinatura'] = np.nan
    sao_pedro_horario_02_rota_sul['zona'] = 'SUL'
    sao_pedro_horario_02_rota_sul['assinatura'] = np.nan
    sao_pedro_horario_02_rota_leste['zona'] = 'LESTE'
    sao_pedro_horario_02_rota_leste['assinatura'] = np.nan
    sao_pedro_horario_02_rota_centro['zona'] = 'CENTRO'
    sao_pedro_horario_02_rota_centro['assinatura'] = np.nan
    sao_pedro_horario_02_rota_sdb['zona'] = 'SDB'
    sao_pedro_horario_02_rota_sdb['assinatura'] = np.nan
    sao_pedro_horario_02_rota_sdc['zona'] = 'SDC'
    sao_pedro_horario_02_rota_sdc['assinatura'] = np.nan
    sao_pedro_horario_02_rota_timon['zona'] = 'TIMON'
    sao_pedro_horario_02_rota_timon['assinatura'] = np.nan
    
    # ORDEM
    sao_pedro_horario_02_sem_rota = sao_pedro_horario_02_sem_rota[ordem]
    sao_pedro_horario_02_rota_sul = sao_pedro_horario_02_rota_sul[ordem]
    sao_pedro_horario_02_rota_norte = sao_pedro_horario_02_rota_norte[ordem]
    sao_pedro_horario_02_rota_leste = sao_pedro_horario_02_rota_leste[ordem]
    sao_pedro_horario_02_rota_centro = sao_pedro_horario_02_rota_centro[ordem]
    sao_pedro_horario_02_rota_sdb = sao_pedro_horario_02_rota_sdb[ordem]
    sao_pedro_horario_02_rota_sdc = sao_pedro_horario_02_rota_sdc[ordem]
    sao_pedro_horario_02_rota_timon = sao_pedro_horario_02_rota_timon[ordem]
    
    #  ARQUIVO sao_pedro HORARIO 02
    
    sao_pedro_rotas_horario_02 = pd.ExcelWriter(titulo_02, engine='xlsxwriter')
    sao_pedro_horario_02_rota_sul.to_excel(sao_pedro_rotas_horario_02, sheet_name='SUL')
    sao_pedro_horario_02_rota_norte.to_excel(sao_pedro_rotas_horario_02, sheet_name='NORTE')
    sao_pedro_horario_02_rota_leste.to_excel(sao_pedro_rotas_horario_02, sheet_name='LESTE')
    sao_pedro_horario_02_rota_centro.to_excel(sao_pedro_rotas_horario_02, sheet_name='CENTRO')
    sao_pedro_horario_02_rota_sdb.to_excel(sao_pedro_rotas_horario_02, sheet_name='SDB')
    sao_pedro_horario_02_rota_sdc.to_excel(sao_pedro_rotas_horario_02, sheet_name='SDC')
    sao_pedro_horario_02_rota_timon.to_excel(sao_pedro_rotas_horario_02, sheet_name='TIMON')
    sao_pedro_horario_02_sem_rota.to_excel(sao_pedro_rotas_horario_02, sheet_name='SEM_ROTA')
    sao_pedro_rotas_horario_02.save()
    
    # SEPARAÇÃO POR ROTA 03

    sao_pedro_horario_03_sem_rota = sao_pedro_horario_03[
        (~sao_pedro_horario_03['bairro'].isin(todos)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_norte = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(norte)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_sul = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(sul)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_leste = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(leste)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_centro = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(centro)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_sdb = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(sdb)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_sdc = sao_pedro_horario_03[
        (sao_pedro_horario_03['bairro'].isin(sdc)) & (sao_pedro_horario_03['cidade'] == 'TERESINA')]
    sao_pedro_horario_03_rota_timon = sao_pedro_horario_03[sao_pedro_horario_03['cidade'] == 'TIMON']
    # CONFERÊNCIA
    soma_rotas = (sao_pedro_horario_03_rota_norte.shape[0], sao_pedro_horario_03_rota_sul.shape[0],
                  sao_pedro_horario_03_rota_leste.shape[0],
                  sao_pedro_horario_03_rota_centro.shape[0], sao_pedro_horario_03_rota_sdb.shape[0],
                  sao_pedro_horario_03_rota_sdc.shape[0],
                  sao_pedro_horario_03_rota_timon.shape[0], sao_pedro_horario_03_sem_rota.shape[0])
    soma_rotas = pd.Series(soma_rotas)
    print(' ', sum(soma_rotas), 'passageiros foram distribuidos em rotas no terceiro horário')
    # DROPAR
    dropar = ['complemento', 'telefone',  'filial']
    sao_pedro_horario_03_sem_rota = sao_pedro_horario_03_sem_rota.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_norte = sao_pedro_horario_03_rota_norte.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_sul = sao_pedro_horario_03_rota_sul.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_leste = sao_pedro_horario_03_rota_leste.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_centro = sao_pedro_horario_03_rota_centro.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_sdb = sao_pedro_horario_03_rota_sdb.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_sdc = sao_pedro_horario_03_rota_sdc.drop(dropar, axis=1)
    sao_pedro_horario_03_rota_timon = sao_pedro_horario_03_rota_timon.drop(dropar, axis=1)

    # ADIÇÃO DE ASSINATURA E SUBTITULOS
    sao_pedro_horario_03_sem_rota['zona'] = 'DESCONHECIDA'
    sao_pedro_horario_03_sem_rota['assinatura'] = np.nan
    sao_pedro_horario_03_rota_norte['zona'] = 'NORTE'
    sao_pedro_horario_03_rota_norte['assinatura'] = np.nan
    sao_pedro_horario_03_rota_sul['zona'] = 'SUL'
    sao_pedro_horario_03_rota_sul['assinatura'] = np.nan
    sao_pedro_horario_03_rota_leste['zona'] = 'LESTE'
    sao_pedro_horario_03_rota_leste['assinatura'] = np.nan
    sao_pedro_horario_03_rota_centro['zona'] = 'CENTRO'
    sao_pedro_horario_03_rota_centro['assinatura'] = np.nan
    sao_pedro_horario_03_rota_sdb['zona'] = 'SDB'
    sao_pedro_horario_03_rota_sdb['assinatura'] = np.nan
    sao_pedro_horario_03_rota_sdc['zona'] = 'SDC'
    sao_pedro_horario_03_rota_sdc['assinatura'] = np.nan
    sao_pedro_horario_03_rota_timon['zona'] = 'TIMON'
    sao_pedro_horario_03_rota_timon['assinatura'] = np.nan

    # ORDEM
    ordem = ['nome', 'rua', 'numero', 'bairro', 'zona', 'horario', 'assinatura', 'status_frequencia']
    sao_pedro_horario_03_sem_rota = sao_pedro_horario_03_sem_rota[ordem]
    sao_pedro_horario_03_rota_sul = sao_pedro_horario_03_rota_sul[ordem]
    sao_pedro_horario_03_rota_norte = sao_pedro_horario_03_rota_norte[ordem]
    sao_pedro_horario_03_rota_leste = sao_pedro_horario_03_rota_leste[ordem]
    sao_pedro_horario_03_rota_centro = sao_pedro_horario_03_rota_centro[ordem]
    sao_pedro_horario_03_rota_sdb = sao_pedro_horario_03_rota_sdb[ordem]
    sao_pedro_horario_03_rota_sdc = sao_pedro_horario_03_rota_sdc[ordem]
    sao_pedro_horario_03_rota_timon = sao_pedro_horario_03_rota_timon[ordem]

    #  ARQUIVO sao_pedro HORARIO 03
    
    sao_pedro_rotas_horario_03 = pd.ExcelWriter(titulo_03, engine='xlsxwriter')
    sao_pedro_horario_03_rota_sul.to_excel(sao_pedro_rotas_horario_03, sheet_name='SUL')
    sao_pedro_horario_03_rota_norte.to_excel(sao_pedro_rotas_horario_03, sheet_name='NORTE')
    sao_pedro_horario_03_rota_leste.to_excel(sao_pedro_rotas_horario_03, sheet_name='LESTE')
    sao_pedro_horario_03_rota_centro.to_excel(sao_pedro_rotas_horario_03, sheet_name='CENTRO')
    sao_pedro_horario_03_rota_sdb.to_excel(sao_pedro_rotas_horario_03, sheet_name='SDB')
    sao_pedro_horario_03_rota_sdc.to_excel(sao_pedro_rotas_horario_03, sheet_name='SDC')
    sao_pedro_horario_03_rota_timon.to_excel(sao_pedro_rotas_horario_03, sheet_name='TIMON')
    sao_pedro_horario_03_sem_rota.to_excel(sao_pedro_rotas_horario_03, sheet_name='SEM ROTA')

    sao_pedro_rotas_horario_03.save()


In [30]:
# BUSCAR FANTASMA
todas_rotas_01 = pd.concat([dirceu_horario_01_rota_sul, dirceu_horario_01_rota_norte,dirceu_horario_01_rota_leste,
                   dirceu_horario_01_rota_centro,dirceu_horario_01_rota_sdb, dirceu_horario_01_rota_sdc,
                   dirceu_horario_01_rota_timon, dirceu_horario_01_sem_rota])  
    
todas_rotas_03 = pd.concat([dirceu_horario_03_rota_sul, dirceu_horario_03_rota_norte,dirceu_horario_03_rota_leste,
                   dirceu_horario_03_rota_centro,dirceu_horario_03_rota_sdb, dirceu_horario_03_rota_sdc,
                   dirceu_horario_03_rota_timon, dirceu_horario_03_sem_rota])
tudo = pd.concat([todas_rotas_01, todas_rotas_03, novos_ativos_inativos])

print(len(todas_rotas_01), len(todas_rotas_03), len(tudo))
tudo['mats'] = tudo.index
print(tudo['mats'].value_counts().tail(20))

9 2 32
254071    2
230148    2
225597    2
239645    2
81782     2
220818    2
237688    2
251321    2
139995    2
185634    2
243125    1
220820    1
139888    1
248476    1
187517    1
170020    1
220808    1
239909    1
220892    1
125571    1
Name: mats, dtype: int64


In [31]:
# REPETIDAS

matriculas = ver_repetidos.index.value_counts()
matriculas.head(10)

197631    1
220818    1
125571    1
170020    1
239909    1
230148    1
220808    1
225597    1
248476    1
139888    1
Name: matricula, dtype: int64

In [32]:
todas_matriculas = novos_ativos_inativos.index
todas_matriculas

Int64Index([220808, 239909, 187517, 139888,  81782, 220818, 220820, 220892,
            251321, 170020, 237688, 239645, 185634, 197631, 125571, 225597,
            230148, 139995, 243125, 254071, 248476],
           dtype='int64', name='matricula')